# IQ-Solver by Luca Scharr
The following aims to demonstrate how to use my IQ-Number-Test-Series solver.  
Simultaneously it was possible to test my solver on data generated by Maximilian Kernbach. The results obtained in this tests are shown below.

In [1]:
#The necessary imports
import os
import sys
import time

import numpy as np
from numpy.random import default_rng
rng = default_rng()

from searchstructure import *
from gen_result import *
from classes import *

import matplotlib.pyplot as plt

### Toy Example
This example shows a problem with the elementwise tests

In [2]:
run = False
if run:
    series = np.asarray([1.,2.,4.,7.,11.,16.])
    pos    = np.asarray([4,5])
    # data(positions to mask, groundtruth, performed positive tests, ancestor, how generated from ancestor)
    p_data = data(pos, series, None, None, None) 
    print('Without elementwise tests:')
    output_p_data = bfs(p_data, 
                        tests = ['const','sum','fac','fib'],
                        cons_tests = ['const','sum','fac'],
                        elem_tests = [])#'prime','cube','square'
    print("\nWoah we are zooming through this!")

    soloutions = return_solutions(output_p_data)
    print('The Groundtruth:\t',series)
    print('The masked positions:\t',pos)
    for soloution in soloutions:
        print('Possible Soloution:\t',soloution.series)
    
    print('\nWith elementwise tests:')
    output_p_data = bfs(p_data, 
                        tests = ['const','sum','fac','fib'],
                        cons_tests = ['const','sum','fac'],
                        elem_tests = ['prime','cube','square'])#'prime','cube','square'
    print("\nWoah we are zooming through this!")

    soloutions = return_solutions(output_p_data)
    print('The Groundtruth:\t',series)
    print('The masked positions:\t',pos)
    for soloution in soloutions:
        print('Possible Soloution:\t',soloution.series)

### Importing the Testsets
Maximilian Kernbach was so nice to let me use these results of him.  
Each testset consists of 1000 Series which all have a length of 11.  
However, I was unable to read the first entry per series, therefore I only used the last 10 entries.  
  
Please ask him if you are allowed to use the test-series.  

The rule for generating is shown here:
<img src="Max_testseries/construction_table.png">

In [3]:
store_len10 = False
if store_len10:
    max_tests0 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic_alternating.csv', delimiter=',', dtype='float')
    max_tests0 = max_tests0.T[1:].T
    max_tests1 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic_geometric_two.csv', delimiter=',', dtype='float')
    max_tests1 = max_tests1.T[1:].T
    max_tests2 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic_geometric.csv', delimiter=',', dtype='float')
    max_tests2 = max_tests2.T[1:].T
    max_tests3 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic_incremental.csv', delimiter=',', dtype='float')
    max_tests3 = max_tests3.T[1:].T
    max_tests4 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic_irrelevant.csv', delimiter=',', dtype='float')
    max_tests4 = max_tests4.T[1:].T
    max_tests5 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic.csv', delimiter=',', dtype='float')
    max_tests5 = max_tests5.T[1:].T
    max_tests6 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_fibonacci.csv', delimiter=',', dtype='float')
    max_tests6 = max_tests6.T[1:].T
    max_tests7 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_geometric.csv', delimiter=',', dtype='float')
    max_tests7 = max_tests7.T[1:].T

In [4]:
store_len7 = False
if store_len7:
    max_tests0_7 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic_alternating.csv', delimiter=',', dtype='float')
    max_tests0_7 = max_tests0.T[1:].T[:7]
    max_tests1_7 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic_geometric_two.csv', delimiter=',', dtype='float')
    max_tests1_7 = max_tests1.T[1:].T[:7]
    max_tests2_7 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic_geometric.csv', delimiter=',', dtype='float')
    max_tests2_7 = max_tests2.T[1:].T[:7]
    max_tests3_7 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic_incremental.csv', delimiter=',', dtype='float')
    max_tests3_7 = max_tests3.T[1:].T[:7]
    max_tests4_7 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic_irrelevant.csv', delimiter=',', dtype='float')
    max_tests4_7 = max_tests4.T[1:].T[:7]
    max_tests5_7 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_arithmetic.csv', delimiter=',', dtype='float')
    max_tests5_7 = max_tests5.T[1:].T[:7]
    max_tests6_7 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_fibonacci.csv', delimiter=',', dtype='float')
    max_tests6_7 = max_tests6.T[1:].T[:7]
    max_tests7_7 = np.genfromtxt('Max_testseries/batchseq_sequences/batchseq_geometric.csv', delimiter=',', dtype='float')
    max_tests7_7 = max_tests7.T[1:].T[:7]

### Performing the Tests
As my task did go beyond finding the next entry in a given series my tests should go aswell.  
I observed that the elementwise tests don't work as expected, therefore each test will be performed twice, once with them and once without them.
The following will be tested:
1. Mask the last entry; how many of the made predictions are correct?
2. Mask 10-50% of the series at random; how many of the predictions are correct now?

This snippet performs the tests. rng.integers(...) as used here can draw positions multiple times.

In [5]:
def perform_test_bfs(test_set, max_masked, perf_all = True, verbose = True):
    start_time = time.time()
    right_answers = 0
    answers = 0
    for truth in test_set:
        pos    = np.asarray([truth.size-1])
        p_data = data(pos, truth, None, None, None)
        output_p_data = bfs(p_data, tests = ['const','sum','fac','fib'],
                            cons_tests = ['const','sum','fac'],
                            elem_tests = [], verbose = False)#'prime','cube','square'
        if np.any(output_p_data) != 0:
            soloutions = return_solutions(output_p_data)
            if np.any(soloutions) != 0:
                for soloution in soloutions:
                    right_answers += all(soloution.series==truth)
                    answers       += 1
            else:
                answers       += 1
        else:
            answers       += 1
    perf_no_elem = right_answers, answers
    end_time = time.time()
    if verbose:
        print("perf_no_elem in: ", end_time-start_time," s")
    if perf_all:
        start_time = time.time()
        right_answers = 0
        answers = 0
        for truth in test_set:
            pos    = np.asarray([truth.size-1])
            p_data = data(pos, truth, None, None, None)
            output_p_data = bfs(p_data, tests = ['const','sum','fac','fib'],
                                cons_tests = ['const','sum','fac'],
                                elem_tests = ['prime','cube','square'], verbose = False)#'prime','cube','square'
            if np.any(output_p_data) != 0:
                soloutions = return_solutions(output_p_data)
                if np.any(soloutions) != 0:
                    for soloution in soloutions:
                        right_answers += all(soloution.series==truth)
                        answers       += 1
                else:
                    answers       += 1
            else:
                answers       += 1
        perf_elem = right_answers, answers
        end_time = time.time()
        if verbose:
            print("perf_elem in: ", end_time-start_time," s")
        #
        perf_no_elem_test = np.zeros((max_masked-1,2))
        perf_elem_test    = np.zeros((max_masked-1,2))
        for i in np.arange(1,max_masked):
            start_time = time.time()
            right_answers = 0
            answers = 0
            for truth in test_set:
                pos    = rng.integers(0, high=truth.size-1, size=(i,))
                p_data = data(pos, truth, None, None, None)
                output_p_data = bfs(p_data, tests = ['const','sum','fac','fib'],
                                    cons_tests = ['const','sum','fac'],
                                    elem_tests = [], verbose = False)#'prime','cube','square'
                if np.any(output_p_data) != 0:
                    soloutions = return_solutions(output_p_data)
                    if np.any(soloutions) != 0:
                        for soloution in soloutions:
                            right_answers += all(soloution.series==truth)
                            answers       += 1
                    else:
                        answers       += 1
                else:
                    answers       += 1
            perf_no_elem_test[i-1] = right_answers, answers
            end_time = time.time()
            if verbose:
                print("perf_no_elem_test[",i-1,"] in: ", end_time-start_time," s")
        for i in np.arange(1,max_masked):
            start_time = time.time()
            right_answers = 0
            answers = 0
            for truth in test_set:
                pos    = rng.integers(0, high=truth.size-1, size=(i,))
                p_data = data(pos, truth, None, None, None)
                output_p_data = bfs(p_data, tests = ['const','sum','fac','fib'],
                                    cons_tests = ['const','sum','fac'],
                                    elem_tests = ['prime','cube','square'], verbose = False)#'prime','cube','square'
                if np.any(output_p_data) != 0:
                    soloutions = return_solutions(output_p_data)
                    if np.any(soloutions) != 0:
                        for soloution in soloutions:
                            right_answers += all(soloution.series==truth)
                            answers       += 1
                    else:
                        answers       += 1
                else:
                    answers       += 1
            perf_elem_test[i-1] = right_answers, answers
            end_time = time.time()
            if verbose:
                print("perf_elem_test[",i-1,"] in: ", end_time-start_time," s")
        return perf_elem, perf_elem_test, perf_no_elem, perf_no_elem_test
    return perf_no_elem

In [6]:
def perform_test_bfs_2(test_set, max_masked, verbose = True):
    perf_no_elem_test = np.zeros((max_masked-1,2))
    for i in np.arange(1,max_masked):
        start_time = time.time()
        right_answers = 0
        answers = 0
        for truth in test_set:
            pos    = rng.choice(truth.size, size=(i,), replace=False)
            p_data = data(pos, truth, None, None, None)
            output_p_data = bfs(p_data, tests = ['const','sum','fac','fib'],
                                cons_tests = ['const','sum','fac'],
                                elem_tests = [], verbose = False)#'prime','cube','square'
            if np.any(output_p_data) != 0:
                soloutions = return_solutions(output_p_data)
                if np.any(soloutions) != 0:
                    for soloution in soloutions:
                        right_answers += all(soloution.series==truth)
                        answers       += 1
                else:
                    answers       += 1
            else:
                answers       += 1
        perf_no_elem_test[i-1] = right_answers, answers
        end_time = time.time()
        if verbose:
            print("perf_no_elem_test[",i-1,"] in: ", end_time-start_time," s")
    perf_no_elem_test_rep = np.zeros((max_masked-1,2))
    for i in np.arange(1,max_masked):
        start_time = time.time()
        right_answers = 0
        answers = 0
        for truth in test_set:
            pos    = rng.integers(0, high=truth.size, size=(i,))
            p_data = data(pos, truth, None, None, None)
            output_p_data = bfs(p_data, tests = ['const','sum','fac','fib'],
                                cons_tests = ['const','sum','fac'],
                                elem_tests = [], verbose = False)#'prime','cube','square'
            if np.any(output_p_data) != 0:
                soloutions = return_solutions(output_p_data)
                if np.any(soloutions) != 0:
                    for soloution in soloutions:
                        right_answers += all(soloution.series==truth)
                        answers       += 1
                else:
                    answers       += 1
            else:
                answers       += 1
        perf_no_elem_test_rep[i-1] = right_answers, answers
        end_time = time.time()
        if verbose:
            print("perf_no_elem_test_rep[",i-1,"] in: ", end_time-start_time," s")
    return perf_no_elem_test, perf_no_elem_test_rep

Here the results are saved:

In [7]:
perform_Test_bfs = False
if perform_Test_bfs:
    perf_elem0, perf_elem_test0, perf_no_elem0, perf_no_elem_test0 = perform_test_bfs(max_tests0, 6)

    perf_elem1, perf_elem_test1, perf_no_elem1, perf_no_elem_test1 = perform_test_bfs(max_tests1, 6)

    perf_elem2, perf_elem_test2, perf_no_elem2, perf_no_elem_test2 = perform_test_bfs(max_tests2, 6)

    perf_elem3, perf_elem_test3, perf_no_elem3, perf_no_elem_test3 = perform_test_bfs(max_tests3, 6)

    perf_elem4, perf_elem_test4, perf_no_elem4, perf_no_elem_test4 = perform_test_bfs(max_tests4, 6)

    perf_elem5, perf_elem_test5, perf_no_elem5, perf_no_elem_test5 = perform_test_bfs(max_tests5, 6)

    perf_elem6, perf_elem_test6, perf_no_elem6, perf_no_elem_test6 = perform_test_bfs(max_tests6, 6)

    perf_elem7, perf_elem_test7, perf_no_elem7, perf_no_elem_test7 = perform_test_bfs(max_tests7, 6)

In [8]:
perform_Test_bfs_2 = False
if perform_Test_bfs_2:
    test0, test0_re = perform_test_bfs_2(max_tests0, 6)

    test1, test1_re = perform_test_bfs_2(max_tests1, 6)

    test2, test2_re = perform_test_bfs_2(max_tests2, 6)

    test3, test3_re = perform_test_bfs_2(max_tests3, 6)

    test4, test4_re = perform_test_bfs_2(max_tests4, 6)

    test5, test5_re = perform_test_bfs_2(max_tests5, 6)

    test6, test6_re = perform_test_bfs_2(max_tests6, 6)

    test7, test7_re = perform_test_bfs_2(max_tests7, 6)

perf_no_elem_test[ 0 ] in:  14.653047323226929  s
perf_no_elem_test[ 1 ] in:  13.504848957061768  s


C:\Users\Luca\Documents\Studium_Master\Lab\IQ-Solver\seq_tests.py:78: RuntimeWarning: invalid value encountered in double_scalars
  quotient_consecutive_elements = (series[indices[1]]/series[indices[0]]) ** (1./differences_indices[0])


perf_no_elem_test[ 2 ] in:  23.021313905715942  s
perf_no_elem_test[ 3 ] in:  67.39168524742126  s
perf_no_elem_test[ 4 ] in:  119.48782634735107  s
perf_no_elem_test_rep[ 0 ] in:  13.051102876663208  s
perf_no_elem_test_rep[ 1 ] in:  13.672623872756958  s
perf_no_elem_test_rep[ 2 ] in:  19.205994844436646  s
perf_no_elem_test_rep[ 3 ] in:  44.93149209022522  s
perf_no_elem_test_rep[ 4 ] in:  78.66675233840942  s


C:\Users\Luca\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


perf_no_elem_test[ 0 ] in:  55.072911739349365  s
perf_no_elem_test[ 1 ] in:  59.59692859649658  s
perf_no_elem_test[ 2 ] in:  58.899169921875  s
perf_no_elem_test[ 3 ] in:  60.55073046684265  s
perf_no_elem_test[ 4 ] in:  64.18943548202515  s
perf_no_elem_test_rep[ 0 ] in:  54.701812744140625  s
perf_no_elem_test_rep[ 1 ] in:  55.80103659629822  s
perf_no_elem_test_rep[ 2 ] in:  57.17062449455261  s
perf_no_elem_test_rep[ 3 ] in:  57.4110152721405  s
perf_no_elem_test_rep[ 4 ] in:  60.95467948913574  s
perf_no_elem_test[ 0 ] in:  29.43142580986023  s
perf_no_elem_test[ 1 ] in:  31.046690940856934  s
perf_no_elem_test[ 2 ] in:  31.761415719985962  s
perf_no_elem_test[ 3 ] in:  35.53565859794617  s
perf_no_elem_test[ 4 ] in:  42.01046538352966  s
perf_no_elem_test_rep[ 0 ] in:  29.13073182106018  s
perf_no_elem_test_rep[ 1 ] in:  30.69299626350403  s
perf_no_elem_test_rep[ 2 ] in:  31.410961627960205  s
perf_no_elem_test_rep[ 3 ] in:  33.51260709762573  s
perf_no_elem_test_rep[ 4 ] in: 

In [9]:
perf_Test_bfs_at_len_7 = False
if perf_Test_bfs_at_len_7:
    test_0_7 = perform_test_bfs(max_tests0_7, 0, perf_all = False)
    test_1_7 = perform_test_bfs(max_tests1_7, 0, perf_all = False)
    test_2_7 = perform_test_bfs(max_tests2_7, 0, perf_all = False)
    test_3_7 = perform_test_bfs(max_tests3_7, 0, perf_all = False)
    test_4_7 = perform_test_bfs(max_tests4_7, 0, perf_all = False)
    test_5_7 = perform_test_bfs(max_tests5_7, 0, perf_all = False)
    test_6_7 = perform_test_bfs(max_tests6_7, 0, perf_all = False)
    test_7_7 = perform_test_bfs(max_tests7_7, 0, perf_all = False)

perf_no_elem in:  0.10396957397460938  s
perf_no_elem in:  0.2811777591705322  s
perf_no_elem in:  0.11624932289123535  s
perf_no_elem in:  0.1926133632659912  s
perf_no_elem in:  0.09459447860717773  s
perf_no_elem in:  0.09185338020324707  s
perf_no_elem in:  0.17257118225097656  s
perf_no_elem in:  0.09112095832824707  s


In [10]:
perf_Test_bfs_at_len_10 = False
if perf_Test_bfs_at_len_10:
    a_test_0 = perform_test_bfs(max_tests0, 0, perf_all = False)
    a_test_1 = perform_test_bfs(max_tests1, 0, perf_all = False)
    a_test_2 = perform_test_bfs(max_tests2, 0, perf_all = False)
    a_test_3 = perform_test_bfs(max_tests3, 0, perf_all = False)
    a_test_4 = perform_test_bfs(max_tests4, 0, perf_all = False)
    a_test_5 = perform_test_bfs(max_tests5, 0, perf_all = False)
    a_test_6 = perform_test_bfs(max_tests6, 0, perf_all = False)
    a_test_7 = perform_test_bfs(max_tests7, 0, perf_all = False)


perf_no_elem in:  13.615203857421875  s
perf_no_elem in:  51.5805721282959  s
perf_no_elem in:  28.73831582069397  s
perf_no_elem in:  40.13708686828613  s
perf_no_elem in:  13.122575521469116  s
perf_no_elem in:  13.353847742080688  s
perf_no_elem in:  25.70992875099182  s
perf_no_elem in:  13.26924180984497  s


In [11]:
overview_needed_bfs = False
if overview_needed_bfs:
    print('Results of test 0:')
    print(perf_elem0, perf_elem_test0, perf_no_elem0, perf_no_elem_test0)

    print('Results of test 1:')
    print(perf_elem1, perf_elem_test1, perf_no_elem1, perf_no_elem_test1)

    print('Results of test 2:')
    print(perf_elem2, perf_elem_test2, perf_no_elem2, perf_no_elem_test2)

    print('Results of test 3:')
    print(perf_elem3, perf_elem_test3, perf_no_elem3, perf_no_elem_test3)

    print('Results of test 4:')
    print(perf_elem4, perf_elem_test4, perf_no_elem4, perf_no_elem_test4)

    print('Results of test 5:')
    print(perf_elem5, perf_elem_test5, perf_no_elem5, perf_no_elem_test5)

    print('Results of test 6:')
    print(perf_elem6, perf_elem_test6, perf_no_elem6, perf_no_elem_test6)

    print('Results of test 7:')
    print(perf_elem7, perf_elem_test7, perf_no_elem7, perf_no_elem_test7)

### The Results
As the elementwise tests seem to need a revisit, lets mainly consider the performance of the algorithm without these tests.  
In the following are plots of the results obtained above.

It can be observed that the precision declines when more positions get masked. This was expected, as the information contained by the series declined.  
Furthermore the performance on the testset Arithmetric Geometric 2 is much worse than the performance on all other testsets. The increased complexity of Arithmetric Geometric 2 seems to be a factor here.  
However the performance on the remaining testsets seems excelent to me, as the precision stays close to 100%.

In [12]:
generate_plots_bfs = False
if generate_plots_bfs:
    coordinates = np.asarray([0,1,2,3,4,5])
    precision = np.asarray([float(perf_no_elem0[0])/perf_no_elem0[1]])
    precision_ = perf_no_elem_test0.astype('float').T[0]/perf_no_elem_test0.T[1]
    precision_[np.isnan(precision_)] = 0.
    height = np.concatenate((precision,precision_))
    height = np.round_(height, decimals=3)

    #To save the results as csv uncomment the next two lines
    #csv_in = np.round_(height.reshape(6,1), decimals=3)
    #np.savetxt('test_arithmetic_alternating.csv', csv_in, delimiter=',')

    #Generate a bar plot
    plt.plot((-.5,6), np.ones((2,)), ls = ':', label = '100% filled correct', color = 'black')
    plt.title('Perecision on the arithmetic alternating Testset')
    plt.bar(coordinates, height, width=0.8, align='center',
            tick_label = ('Last','One','Two','Three','Four','Five'),
            color = 'darkgreen')
    for c,h in zip(coordinates,height):
        if h != 0.:
            plt.text(c, h-.1, h.astype('str'), horizontalalignment='center', color = 'w')
    plt.legend(loc = 'upper right')
    plt.ylabel('Precision')
    plt.xlabel('Masked Positions')
    plt.show()


    coordinates = np.asarray([0,1,2,3,4,5])
    precision = np.asarray([float(perf_elem0[0])/perf_elem0[1]])
    precision_ = perf_elem_test0.astype('float').T[0]/perf_elem_test0.T[1]
    precision_[np.isnan(precision_)] = 0.
    height = np.concatenate((precision,precision_))
    height = np.round_(height, decimals=3)

    #Generate a bar plot
    plt.plot((-.5,6), np.ones((2,)), ls = ':', label = '100% filled correct', color = 'black')
    plt.title('Perecision on the arithmetic alternating Testset')
    plt.bar(coordinates, height, width=0.8, align='center',
            tick_label = ('Last','One','Two','Three','Four','Five'),
            color = 'darkgreen')
    for c,h in zip(coordinates,height):
        if h != 0.:
            plt.text(c, h-.1, h.astype('str'), horizontalalignment='center', color = 'w')
    plt.legend(loc = 'upper right')
    plt.ylabel('Precision')
    plt.xlabel('Masked Positions')
    plt.show()

    coordinates = np.asarray([0,1,2,3,4,5])
    precision = np.asarray([float(perf_no_elem1[0])/perf_no_elem1[1]])
    precision_ = perf_no_elem_test1.astype('float').T[0]/perf_no_elem_test1.T[1]
    precision_[np.isnan(precision_)] = 0.
    height = np.concatenate((precision,precision_))
    height = np.round_(height, decimals=3)


    #csv_in = np.round_(height.reshape(6,1), decimals=3)
    #np.savetxt('test_arithmetic_geometric_2.csv', csv_in, delimiter=',')

    #Generate a bar plot
    plt.plot((-.5,6), np.ones((2,)), ls = ':', label = '100% filled correct', color = 'black')
    plt.title('Perecision on the second arithmetic geometric Testset')
    plt.bar(coordinates, height, width=0.8, align='center',
            tick_label = ('Last','One','Two','Three','Four','Five'),
            color = 'darkgreen')
    for c,h in zip(coordinates,height):
        if h != 0.:
            plt.text(c, h-.1, h.astype('str'), horizontalalignment='center', color = 'w')
    plt.legend(loc = 'upper right')
    plt.ylabel('Precision')
    plt.xlabel('Masked Positions')
    plt.show()

    coordinates = np.asarray([0,1,2,3,4,5])
    precision = np.asarray([float(perf_no_elem2[0])/perf_no_elem2[1]])
    precision_ = perf_no_elem_test2.astype('float').T[0]/perf_no_elem_test2.T[1]
    precision_[np.isnan(precision_)] = 0.
    height = np.concatenate((precision,precision_))
    height = np.round_(height, decimals=3)

    #csv_in = np.round_(height.reshape(6,1), decimals=3)
    #np.savetxt('test_arithmetic_geometric.csv', csv_in, delimiter=',')

    #Generate a bar plot
    plt.plot((-.5,6), np.ones((2,)), ls = ':', label = '100% filled correct', color = 'black')
    plt.title('Perecision on the first arithmetic geometric Testset')
    plt.bar(coordinates, height, width=0.8, align='center',
            tick_label = ('Last','One','Two','Three','Four','Five'),
            color = 'darkgreen')
    for c,h in zip(coordinates,height):
        if h != 0.:
            plt.text(c, h-.1, h.astype('str'), horizontalalignment='center', color = 'w')
    plt.legend(loc = 'upper right')
    plt.ylabel('Precision')
    plt.xlabel('Masked Positions')
    plt.show()

    coordinates = np.asarray([0,1,2,3,4,5])
    precision = np.asarray([float(perf_no_elem3[0])/perf_no_elem3[1]])
    precision_ = perf_no_elem_test3.astype('float').T[0]/perf_no_elem_test3.T[1]
    precision_[np.isnan(precision_)] = 0.
    height = np.concatenate((precision,precision_))
    height = np.round_(height, decimals=3)

    #csv_in = np.round_(height.reshape(6,1), decimals=3)
    #np.savetxt('test_arithmetic_incremental.csv', csv_in, delimiter=',')

    #Generate a bar plot
    plt.plot((-.5,6), np.ones((2,)), ls = ':', label = '100% filled correct', color = 'black')
    plt.title('Perecision on the arithmetic incremental Testset')
    plt.bar(coordinates, height, width=0.8, align='center',
            tick_label = ('Last','One','Two','Three','Four','Five'),
            color = 'darkgreen')
    for c,h in zip(coordinates,height):
        if h != 0.:
            plt.text(c, h-.1, h.astype('str'), horizontalalignment='center', color = 'w')
    plt.legend(loc = 'upper right')
    plt.ylabel('Precision')
    plt.xlabel('Masked Positions')
    plt.show()

    coordinates = np.asarray([0,1,2,3,4,5])
    precision = np.asarray([float(perf_no_elem4[0])/perf_no_elem4[1]])
    precision_ = perf_no_elem_test4.astype('float').T[0]/perf_no_elem_test4.T[1]
    precision_[np.isnan(precision_)] = 0.
    height = np.concatenate((precision,precision_))
    height = np.round_(height, decimals=3)

    #csv_in = np.round_(height.reshape(6,1), decimals=3)
    #np.savetxt('test_arithmetic_irrelevant.csv', csv_in, delimiter=',')

    #Generate a bar plot
    plt.plot((-.5,6), np.ones((2,)), ls = ':', label = '100% filled correct', color = 'black')
    plt.title('Perecision on the arithmetic irrelevant Testset')
    plt.bar(coordinates, height, width=0.8, align='center',
            tick_label = ('Last','One','Two','Three','Four','Five'),
            color = 'darkgreen')
    for c,h in zip(coordinates,height):
        if h != 0.:
            plt.text(c, h-.1, h.astype('str'), horizontalalignment='center', color = 'w')
    plt.legend(loc = 'upper right')
    plt.ylabel('Precision')
    plt.xlabel('Masked Positions')
    plt.show()

    coordinates = np.asarray([0,1,2,3,4,5])
    precision = np.asarray([float(perf_no_elem5[0])/perf_no_elem5[1]])
    precision_ = perf_no_elem_test5.astype('float').T[0]/perf_no_elem_test5.T[1]
    precision_[np.isnan(precision_)] = 0.
    height = np.concatenate((precision,precision_))
    height = np.round_(height, decimals=3)

    #csv_in = np.round_(height.reshape(6,1), decimals=3)
    #np.savetxt('test_arithmetic.csv', csv_in, delimiter=',')

    #Generate a bar plot
    plt.plot((-.5,6), np.ones((2,)), ls = ':', label = '100% filled correct', color = 'black')
    plt.title('Perecision on the arithmetic Testset')
    plt.bar(coordinates, height, width=0.8, align='center',
            tick_label = ('Last','One','Two','Three','Four','Five'),
            color = 'darkgreen')
    for c,h in zip(coordinates,height):
        if h != 0.:
            plt.text(c, h-.1, h.astype('str'), horizontalalignment='center', color = 'w')
    plt.legend(loc = 'upper right')
    plt.ylabel('Precision')
    plt.xlabel('Masked Positions')
    plt.show()

    coordinates = np.asarray([0,1,2,3,4,5])
    precision = np.asarray([float(perf_no_elem6[0])/perf_no_elem6[1]])
    precision_ = perf_no_elem_test6.astype('float').T[0]/perf_no_elem_test6.T[1]
    precision_[np.isnan(precision_)] = 0.
    height = np.concatenate((precision,precision_))
    height = np.round_(height, decimals=3)

    #csv_in = np.round_(height.reshape(6,1), decimals=3)
    #np.savetxt('test_fibonacci.csv', csv_in, delimiter=',')

    #Generate a bar plot
    plt.plot((-.5,6), np.ones((2,)), ls = ':', label = '100% filled correct', color = 'black')
    plt.title('Perecision on the fibonacci Testset')
    plt.bar(coordinates, height, width=0.8, align='center',
            tick_label = ('Last','One','Two','Three','Four','Five'),
            color = 'darkgreen')
    for c,h in zip(coordinates,height):
        if h != 0.:
            plt.text(c, h-.1, h.astype('str'), horizontalalignment='center', color = 'w')
    plt.legend(loc = 'upper right')
    plt.ylabel('Precision')
    plt.xlabel('Masked Positions')
    plt.show()

    coordinates = np.asarray([0,1,2,3,4,5])
    precision = np.asarray([float(perf_no_elem7[0])/perf_no_elem7[1]])
    precision_ = perf_no_elem_test7.astype('float').T[0]/perf_no_elem_test7.T[1]
    precision_[np.isnan(precision_)] = 0.
    height = np.concatenate((precision,precision_))
    height = np.round_(height, decimals=3)

    #csv_in = np.round_(height.reshape(6,1), decimals=3)
    #np.savetxt('test_geometric.csv', csv_in, delimiter=',')

    #Generate a bar plot
    plt.plot((-.5,6), np.ones((2,)), ls = ':', label = '100% filled correct', color = 'black')
    plt.title('Perecision on the geometric Testset')
    plt.bar(coordinates, height, width=0.8, align='center',
            tick_label = ('Last','One','Two','Three','Four','Five'),
            color = 'darkgreen')
    for c,h in zip(coordinates,height):
        if h != 0.:
            plt.text(c, h-.1, h.astype('str'), horizontalalignment='center', color = 'w')
    plt.legend(loc = 'upper right')
    plt.ylabel('Precision')
    plt.xlabel('Masked Positions')
    plt.show()

In [13]:
def gen_precision_tests(test0):
    precision_no_re0 = test0.astype('float').T[0]/test0.T[1]
    precision_no_re0[np.isnan(precision_no_re0)] = 0.
    precision_no_re0 = np.round_(precision_no_re0, decimals=3)
    return precision_no_re0

def gen_precision_tests_1d(test0):
    precision_no_re0 = np.asarray([float(test0[0])/test0[1]])
    precision_no_re0[np.isnan(precision_no_re0)] = 0.
    precision_no_re0 = np.round_(precision_no_re0, decimals=3)
    return precision_no_re0

In [14]:
gen_res_precision_tests_len_10 = False
if gen_res_precision_tests_len_10:
    precision_no_re0 = gen_precision_tests(test0)
    np.savetxt('arithmetic_alternating_no_replacement.csv', precision_no_re0, delimiter=',')
    precision_re0 = gen_precision_tests(test0_re)
    np.savetxt('arithmetic_alternating_replacement.csv', precision_re0, delimiter=',')

    precision_no_re1 = gen_precision_tests(test1)
    np.savetxt('arithmetic_geometric_2_no_replacement.csv', precision_no_re1, delimiter=',')
    precision_re1 = gen_precision_tests(test1_re)
    np.savetxt('arithmetic_geometric_2_replacement.csv', precision_re1, delimiter=',')

    precision_no_re2 = gen_precision_tests(test2)
    np.savetxt('arithmetic_geometric_no_replacement.csv', precision_no_re2, delimiter=',')
    precision_re2 = gen_precision_tests(test2_re)
    np.savetxt('arithmetic_geometric_replacement.csv', precision_re2, delimiter=',')

    precision_no_re3 = gen_precision_tests(test3)
    np.savetxt('arithmetic_incremental_no_replacement.csv', precision_no_re3, delimiter=',')
    precision_re3 = gen_precision_tests(test3_re)
    np.savetxt('arithmetic_incremental_replacement.csv', precision_re3, delimiter=',')

    precision_no_re4 = gen_precision_tests(test4)
    np.savetxt('arithmetic_irrelevant_no_replacement.csv', precision_no_re4, delimiter=',')
    precision_re4 = gen_precision_tests(test4_re)
    np.savetxt('arithmetic_irrelevant_replacement.csv', precision_re4, delimiter=',')

    precision_no_re5 = gen_precision_tests(test5)
    np.savetxt('arithmetic_no_replacement.csv', precision_no_re5, delimiter=',')
    precision_re5 = gen_precision_tests(test5_re)
    np.savetxt('arithmetic_replacement.csv', precision_re5, delimiter=',')

    precision_no_re6 = gen_precision_tests(test6)
    np.savetxt('fibonacci_no_replacement.csv', precision_no_re6, delimiter=',')
    precision_re6 = gen_precision_tests(test6_re)
    np.savetxt('fibonacci_replacement.csv', precision_re6, delimiter=',')

    precision_no_re7 = gen_precision_tests(test7)
    np.savetxt('geometric_no_replacement.csv', precision_no_re7, delimiter=',')
    precision_re7 = gen_precision_tests(test7_re)
    np.savetxt('geometric_replacement.csv', precision_re7, delimiter=',')

In [15]:
gen_res_precision_tests_len_7 = False
if gen_res_precision_tests_len_7:
    precision_no_re0 = gen_precision_tests_1d(test_0_7)
    np.savetxt('arithmetic_alternating_7.csv', precision_no_re0, delimiter=',')
    
    precision_re1 = gen_precision_tests_1d(test_1_7)
    np.savetxt('arithmetic_geometric_2_7.csv', precision_re1, delimiter=',')

    precision_re2 = gen_precision_tests_1d(test_2_7)
    np.savetxt('arithmetic_geometric_7.csv', precision_re2, delimiter=',')

    precision_re3 = gen_precision_tests_1d(test_3_7)
    np.savetxt('arithmetic_incremental_7.csv', precision_re3, delimiter=',')

    precision_re4 = gen_precision_tests_1d(test_4_7)
    np.savetxt('arithmetic_irrelevant_7.csv', precision_re4, delimiter=',')

    precision_re5 = gen_precision_tests_1d(test_5_7)
    np.savetxt('arithmetic_7.csv', precision_re5, delimiter=',')

    precision_re6 = gen_precision_tests_1d(test_6_7)
    np.savetxt('fibonacci_7.csv', precision_re6, delimiter=',')

    precision_re7 = gen_precision_tests_1d(test_7_7)
    np.savetxt('geometric_7.csv', precision_re7, delimiter=',')